# Import Library

In [ ]:
import mne
import numpy as np
import time
import os
import os.path as op
import matplotlib.pyplot as plt
import nibabel as nib
from mne.datasets import sample
from mne.minimum_norm import make_inverse_operator, apply_inverse_epochs, apply_inverse
from mne.datasets import fetch_fsaverage
import scipy.io
from scipy.io import loadmat
from scipy.spatial import Delaunay
from scipy import stats
import PIL
from PIL import Image
import datetime
import tensorflow as tf
from tensorflow import keras
from keras.preprocessing.image import ImageDataGenerator
from keras import Sequential
from keras.layers import Conv2D, DepthwiseConv2D, SeparableConv2D, MaxPool2D, AveragePooling2D, GlobalAveragePooling2D, Dense, Activation, Flatten, Concatenate, BatchNormalization, Dropout, Input, Conv1D, ReLU, MaxPooling1D
from keras.constraints import max_norm
from keras.layers.merge import concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import plot_model
from tensorboard.backend.event_processing import event_accumulator
import pandas as pd
# Load the TensorBoard notebook extension
%load_ext tensorboard
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, cohen_kappa_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import StandardScaler
import gc
import json
import multiprocessing
from scipy.spatial import KDTree
from nibabel.nifti1 import Nifti1Image
from nilearn import plotting

%matplotlib inline
#%matplotlib qt

DIRECTORY_PATH = os.getcwd()
EXTERNAL_STORAGE_PATH = "E:\Motor Imagery"
RECONSTRUCT_SAVE_FOLDER = "v3-v5 region"
n_splits = 5

# force tensorflow to use cpu when facing memory issue
# os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# Preprocess Data

In [ ]:
img = nib.load("/Users/ivanlim/Downloads/MRIcron_windows/MRIcron/Resources/templates/brodmann.nii.gz")
ch2_img = nib.load("C:/Users/ivanlim/Downloads/MRIcron_windows/MRIcron/Resources/templates/ch2.nii.gz")

brodmann_data = img.get_fdata()
"""
motor
"""
# Area 4– Primary motor cortex
"""
visual
"""
# Area 17 – Primary visual cortex (V1)
# Area 18 – Secondary visual cortex (V2)
# Area 19 – Associative visual cortex (V3, V4, V5)
# Area 20 – Inferior temporal gyrus
# Area 21 – Middle temporal gyrus
# Area 22 – Part of the superior temporal gyrus, included in Wernicke's area
# Area 37 – Fusiform gyrus
brodmann_visual = []
selected_area = [17, 18, 19, 20, 21, 22, 37]
#selected_area = [17, 18, 19]

# old: compute one convex hull for all the selected regions
# new: compute a single convex hull for one selected region then combine them together
reconstruct_mode = "new"

for area in selected_area:
    if reconstruct_mode == "old":
        if len(brodmann_visual) == 0:
            brodmann_visual.append(brodmann_data.reshape(-1) == area)
        else:
            brodmann_visual[0] += brodmann_data.reshape(-1) == area
    else:
        brodmann_visual.append(brodmann_data.reshape(-1) == area)

print(brodmann_visual)
print("brodmann template shape: " + str(brodmann_data.shape))
if reconstruct_mode == "old":
    print("chosen points: " + str(np.sum(brodmann_visual[0])))
else:
    chosen_points = None
    for selected_region in brodmann_visual:
        print(np.sum(selected_region))
        if chosen_points is None:
            chosen_points = np.array(selected_region, copy=True)
        else:
            chosen_points += selected_region
    print("chosen points: " + str(np.sum(chosen_points)))
        
shape, affine = img.shape[:3], img.affine
coords = np.array(np.meshgrid(*(range(i) for i in shape), indexing='ij'))
coords = np.rollaxis(coords, 0, len(shape) + 1)
mm_coords = nib.affines.apply_affine(affine, coords)

def in_hull(p, hull):
    """
    Test if points in `p` are in `hull`

    `p` should be a `NxK` coordinates of `N` points in `K` dimensions
    `hull` is either a scipy.spatial.Delaunay object or the `MxK` array of the 
    coordinates of `M` points in `K`dimensions for which Delaunay triangulation
    will be computed
    """
    if not isinstance(hull,Delaunay):
        hull = Delaunay(hull)

    return hull.find_simplex(p)>=0

my_left_points = None
my_right_points = None

In [ ]:
# cd to google drive
os.chdir("G:")

# Download fsaverage files
fs_dir = fetch_fsaverage(verbose=True)
subjects_dir = op.dirname(fs_dir)

# The files live in:
mne_subject = 'fsaverage'
trans = 'fsaverage'  # MNE has a built-in fsaverage transformation
src = op.join(fs_dir, 'bem', 'fsaverage-ico-5-src.fif')
bem = op.join(fs_dir, 'bem', 'fsaverage-5120-5120-5120-bem-sol.fif')

source = mne.read_source_spaces(src)
left = source[0]
right = source[1]
left_pos = left["rr"][left["inuse"]==1]
right_pos = right["rr"][right["inuse"]==1]
                        
transformation = mne.read_trans(op.join(fs_dir, "bem", "fsaverage-trans.fif"))

save_path = op.join(os.getcwd(), "Shared drives", "Motor Imagery", "Source Estimate")

In [ ]:
"""
create mne epochs data structure from numpy array
merge training and evaluation data
"""
def create_epochs(data_path):
    subjects_data = {}
    files = os.listdir(data_path)
    for file in files:
        # load data
        data = loadmat(op.join(data_path, file))
        sampling_freq = data['Fs']
        labels = data['categoryLabels'].reshape(-1)     # trials
        epochs = data['X_3D']                           # no of channels, time, trials
        epochs = np.moveaxis(epochs, [0, 1], [1, 2])    # trials, no of channels, time

        # create info
        GSN_128 = mne.channels.make_standard_montage('GSN-HydroCel-128')
        ch_names = GSN_128.ch_names[:124]
        ch_types = ['eeg'] * 124
        info = mne.create_info(ch_names, ch_types=ch_types, sfreq=sampling_freq)
        info.set_montage(GSN_128)

        # create epochs
        epochs = mne.EpochsArray(epochs, info, verbose=False)
        
        # Set common average reference
        epochs.set_eeg_reference('average', projection=True, verbose=False)
        
        subjects_data[file[:-4]] = {}
        subjects_data[file[:-4]]["epochs"] = epochs
        subjects_data[file[:-4]]["labels"] = labels

    return subjects_data


"""

"""
def create_model(model_name="default"):
    if model_name == "default":
        model = tf.keras.models.Sequential([
            Conv1D(filters=200, kernel_size=3, strides=1, padding='same'),
            BatchNormalization(),
            Dropout(0.5),
            ReLU(),
            Flatten(),
            Dense(6, activation="softmax")
        ])
    elif model_name == "eegnet":
        model = tf.keras.models.Sequential([
                Conv2D(16, (1, 8), use_bias = False, activation = 'linear', padding='same', name = 'Spectral_filter'),
                BatchNormalization(),
                DepthwiseConv2D((124, 1), use_bias = False, padding='valid', depth_multiplier = 2, activation = 'linear',
                depthwise_constraint = tf.keras.constraints.MaxNorm(max_value=1), name = 'Spatial_filter'),
                BatchNormalization(),
                Activation('elu'),
                AveragePooling2D((1, 2)),
                Dropout(0.5),
                SeparableConv2D(32, (1, 4), use_bias = False, activation = 'linear', padding = 'same'),
                BatchNormalization(),
                Activation('elu'),
                AveragePooling2D((1, 2)),
                Dropout(0.5),
                Flatten(),
                Dense(6, activation = 'softmax', kernel_constraint = max_norm(0.25))
            ])

    return model

In [ ]:
"""
Create source activity and reconstructed eeg respectively for each subject

For each subject, there are six events in total, i.e. 
(1=Human Body; 2=Human Face; 3=Animal Body; 4=Animal Face; 5=Fruit Vegetable; 6=Inanimate Object) 
Split these data into train and test set using kfold
Compute the noise covariance matrix on train set and apply it to test set
Create source activity (only visual region) first by applying an inverse operator to the epochs 
Create reconstructed eeg by applying a forward operator to the source activity acquired earlier
Save both these files to disk
"""
def apply_inverse_and_forward_kfold(epochs, n_splits=5, save_inverse=True, save_forward=True, subjects=None):
    global my_left_points, my_right_points
    
    if subjects is None:
        subjects = epochs.keys()
    
    for subject in subjects:  
        X, Y = [], []
        info = None
        counter = 0
        
        X = epochs[subject]["epochs"].get_data()
        print(X.shape)
        Y = epochs[subject]["labels"]
        print(Y.shape)
        info = epochs[subject]["epochs"].info
        
        X = np.array(X)
        Y = np.array(Y)
        
        skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
        for train_index, test_index in skf.split(X, Y):
            counter += 1
            X_train, X_test = X[train_index], X[test_index]
            Y_train, Y_test = Y[train_index], Y[test_index]
            
            X_train = mne.EpochsArray(X_train, info, verbose=False)
            X_test = mne.EpochsArray(X_test, info, verbose=False)
            
            noise_cov = mne.compute_covariance(X_train, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)
            fwd = mne.make_forward_solution(info, trans=trans, src=src,
                            bem=bem, eeg=True, meg=False, mindist=5.0, n_jobs=1, verbose=False)
            fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                         use_cps=True, verbose=False)
            leadfield = fwd_fixed['sol']['data']
            inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)
            
            method = "sLORETA"
            snr = 3.
            lambda2 = 1. / snr ** 2
            stc_train = apply_inverse_epochs(X_train, inverse_operator, lambda2,
                                          method=method, pick_ori="normal", verbose=True)
            
            # get visual region points (once)
            if my_left_points is None and my_right_points is None:
                my_source = stc_train[0]
                mni_lh = mne.vertex_to_mni(my_source.vertices[0], 0, mne_subject)
                #print(mni_lh.shape)
                mni_rh = mne.vertex_to_mni(my_source.vertices[1], 1, mne_subject)
                #print(mni_rh.shape)

                fig = plt.figure(figsize=(8, 8))
                ax = fig.add_subplot(projection='3d')
                for selected_region in brodmann_visual:
                    ax.scatter(mm_coords.reshape(-1, 3)[selected_region][:, 0], mm_coords.reshape(-1, 3)[selected_region][:, 1], mm_coords.reshape(-1, 3)[selected_region][:, 2], s=15, marker='|')
                ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='_')
                ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
                ax.set_xlabel('X')
                ax.set_ylabel('Y')
                ax.set_zlabel('Z')
                plt.show()

                my_left_points = None
                my_right_points = None
                for selected_region in brodmann_visual:
                    print(np.sum(selected_region))
                    if my_left_points is None:
                        my_left_points = in_hull(mni_lh, mm_coords.reshape(-1, 3)[selected_region])
                        my_right_points = in_hull(mni_rh, mm_coords.reshape(-1, 3)[selected_region])
                    else:
                        my_left_points += in_hull(mni_lh, mm_coords.reshape(-1, 3)[selected_region])
                        my_right_points += in_hull(mni_rh, mm_coords.reshape(-1, 3)[selected_region])

                mni_left_visual = mne.vertex_to_mni(my_source.vertices[0][my_left_points], 0, mne_subject)
                print(mni_left_visual.shape)
                mni_right_visual = mne.vertex_to_mni(my_source.vertices[1][my_right_points], 1, mne_subject)
                print(mni_right_visual.shape)

                fig = plt.figure(figsize=(8, 8))
                ax = fig.add_subplot(projection='3d')
                ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='|')
                ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
                ax.scatter(mni_left_visual[:, 0], mni_left_visual[:, 1], mni_left_visual[:, 2], s=15, marker='o')
                ax.scatter(mni_right_visual[:, 0], mni_right_visual[:, 1], mni_right_visual[:, 2], s=15, marker='^')
                ax.set_xlabel('X')
                ax.set_ylabel('Y')
                ax.set_zlabel('Z')
                plt.show()
                
            print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)
            #print(stc_train[0].data.shape)
            
            # train set
            # slice source activity data
            left_hemi_data = []
            right_hemi_data = []
            for source in stc_train:
                left_hemi_data.append(source.data[:len(source.vertices[0])][my_left_points])
                right_hemi_data.append(source.data[-len(source.vertices[1]):][my_right_points])
            left_hemi_data = np.array(left_hemi_data)
            right_hemi_data = np.array(right_hemi_data)
            if save_inverse:
                source_activity_path = op.join(EXTERNAL_STORAGE_PATH, RECONSTRUCT_SAVE_FOLDER, "data", "source activity", subject)
                if not op.exists(source_activity_path):
                    os.makedirs(source_activity_path)
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_train_X.npz"), data=np.append(left_hemi_data, right_hemi_data, axis=1))
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_train_Y.npz"), data=Y_train)
            # slice reconstructed eeg data
            reconstructed_eeg_data = []
            for source in stc_train:
                visual_source = np.zeros_like(source.data)
                visual_source[:len(source.vertices[0])][my_left_points] = source.data[:len(source.vertices[0])][my_left_points]
                visual_source[-len(source.vertices[1]):][my_right_points] = source.data[-len(source.vertices[1]):][my_right_points]
                visual_eeg = np.dot(leadfield, visual_source)
                reconstructed_eeg_data.append(visual_eeg)
            if save_forward:
                reconstructed_eeg_path = op.join(EXTERNAL_STORAGE_PATH, RECONSTRUCT_SAVE_FOLDER, "data", "reconstructed eeg", subject)
                if not op.exists(reconstructed_eeg_path):
                    os.makedirs(reconstructed_eeg_path)
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_train_X.npz"), data=np.array(reconstructed_eeg_data))
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_train_Y.npz"), data=Y_train)
            
            del stc_train
            gc.collect()
            
            stc_test = apply_inverse_epochs(X_test, inverse_operator, lambda2,
                              method=method, pick_ori="normal", verbose=True)
            # test set
            # slice source activity data
            left_hemi_data = []
            right_hemi_data = []
            for source in stc_test:
                left_hemi_data.append(source.data[:len(source.vertices[0])][my_left_points])
                right_hemi_data.append(source.data[-len(source.vertices[1]):][my_right_points])
            left_hemi_data = np.array(left_hemi_data)
            right_hemi_data = np.array(right_hemi_data)
            if save_inverse:
                source_activity_path = op.join(EXTERNAL_STORAGE_PATH, RECONSTRUCT_SAVE_FOLDER, "data", "source activity", subject)
                if not op.exists(source_activity_path):
                    os.makedirs(source_activity_path)
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_test_X.npz"), data=np.append(left_hemi_data, right_hemi_data, axis=1))
                np.savez_compressed(op.join(source_activity_path, str(counter)+"_test_Y.npz"), data=Y_test)
            # slice reconstructed eeg data
            reconstructed_eeg_data = []
            for source in stc_test:
                visual_source = np.zeros_like(source.data)
                visual_source[:len(source.vertices[0])][my_left_points] = source.data[:len(source.vertices[0])][my_left_points]
                visual_source[-len(source.vertices[1]):][my_right_points] = source.data[-len(source.vertices[1]):][my_right_points]
                visual_eeg = np.dot(leadfield, visual_source)
                reconstructed_eeg_data.append(visual_eeg)
            if save_forward:
                reconstructed_eeg_path = op.join(EXTERNAL_STORAGE_PATH, RECONSTRUCT_SAVE_FOLDER, "data", "reconstructed eeg", subject)
                if not op.exists(reconstructed_eeg_path):
                    os.makedirs(reconstructed_eeg_path)
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_test_X.npz"), data=np.array(reconstructed_eeg_data))
                np.savez_compressed(op.join(reconstructed_eeg_path, str(counter)+"_test_Y.npz"), data=Y_test)
            
            del X_train, X_test, Y_train, Y_test
            del stc_test, reconstructed_eeg_data, left_hemi_data, right_hemi_data
            gc.collect()
               
def get_inverse_and_forward_information(epochs):
    
    subject = "S1"
    X, Y = [], []
    info = None
    
    X = epochs[subject]["epochs"].get_data()
    print(X.shape)
    Y = epochs[subject]["labels"]
    print(Y.shape)
    info = epochs[subject]["epochs"].info

    X = np.array(X)
    Y = np.array(Y)
    X_epochs = mne.EpochsArray(X, info, verbose=False)
    X_evoked = X_epochs.average().pick("eeg")

    noise_cov = mne.compute_covariance(X_epochs, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)
    fwd = mne.make_forward_solution(info, trans=trans, src=src,
                    bem=bem, eeg=True, meg=False, mindist=5.0, n_jobs=1, verbose=False)
    fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                 use_cps=True, verbose=False)
    leadfield = fwd_fixed['sol']['data']
    inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)

    method = "sLORETA"
    snr = 3.
    lambda2 = 1. / snr ** 2
    stc = apply_inverse(X_evoked, inverse_operator, lambda2, method=method, pick_ori="normal", verbose=True)

    # get visual region points
    my_source = stc
    mni_lh = mne.vertex_to_mni(my_source.vertices[0], 0, mne_subject)
    print(mni_lh.shape)
    mni_rh = mne.vertex_to_mni(my_source.vertices[1], 1, mne_subject)
    print(mni_rh.shape)

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    for selected_region in brodmann_visual:
        ax.scatter(mm_coords.reshape(-1, 3)[selected_region][:, 0], mm_coords.reshape(-1, 3)[selected_region][:, 1], mm_coords.reshape(-1, 3)[selected_region][:, 2], s=15, marker='|')
    ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='_')
    ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

    my_left_points = None
    my_right_points = None
    for selected_region in brodmann_visual:
        print(np.sum(selected_region))
        if my_left_points is None:
            my_left_points = in_hull(mni_lh, mm_coords.reshape(-1, 3)[selected_region])
            my_right_points = in_hull(mni_rh, mm_coords.reshape(-1, 3)[selected_region])
        else:
            my_left_points += in_hull(mni_lh, mm_coords.reshape(-1, 3)[selected_region])
            my_right_points += in_hull(mni_rh, mm_coords.reshape(-1, 3)[selected_region])

    mni_left_visual = mne.vertex_to_mni(my_source.vertices[0][my_left_points], 0, mne_subject)
    print(mni_left_visual.shape)
    mni_right_visual = mne.vertex_to_mni(my_source.vertices[1][my_right_points], 1, mne_subject)
    print(mni_right_visual.shape)

    fig = plt.figure(figsize=(8, 8))
    ax = fig.add_subplot(projection='3d')
    ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='|')
    ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
    ax.scatter(mni_left_visual[:, 0], mni_left_visual[:, 1], mni_left_visual[:, 2], s=15, marker='o')
    ax.scatter(mni_right_visual[:, 0], mni_right_visual[:, 1], mni_right_visual[:, 2], s=15, marker='^')
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show()

    print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)
    print(stc.data.shape)

    information = {"my_left_points": my_left_points, 
                   "my_right_points": my_right_points, 
                   "stc_data_shape": stc.data.shape, 
                   "leadfield": leadfield,
                   "left_vertices": stc.vertices[0],
                   "right_vertices": stc.vertices[1],
                   "inverse_operator": inverse_operator}

    return information

"""
plot original data evoked topomap and reconstruct data topomap
original data -> source activity (only visual) -> reconstruct eeg
"""
def plot_evoked_topomap(epochs):
    global my_left_points, my_right_points
    for subject in epochs.keys():  
        X, Y = [], []
        info = None
        counter = 0

        X = epochs[subject]["epochs"]
        print(X.get_data().shape)
        Y = epochs[subject]["labels"]
        print(Y.shape)
        info = epochs[subject]["epochs"].info

        noise_cov = mne.compute_covariance(X, tmax=0., method=['shrunk', 'empirical'], rank=None, verbose=False)
        fwd = mne.make_forward_solution(info, trans=trans, src=src,
                        bem=bem, eeg=True, meg=False, mindist=5.0, n_jobs=1, verbose=False)
        fwd_fixed = mne.convert_forward_solution(fwd, surf_ori=True, force_fixed=True,
                                     use_cps=True, verbose=False)
        leadfield = fwd_fixed['sol']['data']
        inverse_operator = make_inverse_operator(info, fwd, noise_cov, loose=0.2, depth=0.8, verbose=False)

        method = "sLORETA"
        snr = 3.
        lambda2 = 1. / snr ** 2
        stc = apply_inverse_epochs(X, inverse_operator, lambda2,
                                      method=method, pick_ori="normal", verbose=True)

        # get visual region points (once)
        if my_left_points is None and my_right_points is None:
            my_source = stc[0]
            mni_lh = mne.vertex_to_mni(my_source.vertices[0], 0, mne_subject)
            print(mni_lh.shape)
            mni_rh = mne.vertex_to_mni(my_source.vertices[1], 1, mne_subject)
            print(mni_rh.shape)

            """
            fig = plt.figure(figsize=(8, 8))
            ax = fig.add_subplot(projection='3d')
            for selected_region in brodmann_visual:
                ax.scatter(mm_coords.reshape(-1, 3)[selected_region][:, 0], mm_coords.reshape(-1, 3)[selected_region][:, 1], mm_coords.reshape(-1, 3)[selected_region][:, 2], s=15, marker='|')
            ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='_')
            ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')
            plt.show()
            """
            
            my_left_points = None
            my_right_points = None
            for selected_region in brodmann_visual:
                if my_left_points is None:
                    my_left_points = in_hull(mni_lh, mm_coords.reshape(-1, 3)[selected_region])
                    my_right_points = in_hull(mni_rh, mm_coords.reshape(-1, 3)[selected_region])
                else:
                    my_left_points += in_hull(mni_lh, mm_coords.reshape(-1, 3)[selected_region])
                    my_right_points += in_hull(mni_rh, mm_coords.reshape(-1, 3)[selected_region])

            mni_left_visual = mne.vertex_to_mni(my_source.vertices[0][my_left_points], 0, mne_subject)
            print(mni_left_visual.shape)
            mni_right_visual = mne.vertex_to_mni(my_source.vertices[1][my_right_points], 1, mne_subject)
            print(mni_right_visual.shape)

            """
            fig = plt.figure(figsize=(8, 8))
            ax = fig.add_subplot(projection='3d')
            ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='|')
            ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
            ax.scatter(mni_left_visual[:, 0], mni_left_visual[:, 1], mni_left_visual[:, 2], s=15, marker='o')
            ax.scatter(mni_right_visual[:, 0], mni_right_visual[:, 1], mni_right_visual[:, 2], s=15, marker='^')
            ax.set_xlabel('X')
            ax.set_ylabel('Y')
            ax.set_zlabel('Z')
            plt.show()
            """

        #print("Leadfield size : %d sensors x %d dipoles" % leadfield.shape)
        #print(stc_train[0].data.shape)

        # slice reconstructed eeg data
        reconstructed_eeg_data = []
        for source in stc:
            visual_source = np.zeros_like(source.data)
            visual_source[:len(source.vertices[0])][my_left_points] = source.data[:len(source.vertices[0])][my_left_points]
            visual_source[-len(source.vertices[1]):][my_right_points] = source.data[-len(source.vertices[1]):][my_right_points]
            visual_eeg = np.dot(leadfield, visual_source)
            reconstructed_eeg_data.append(visual_eeg)

        reconstructed_eeg_data = np.array(reconstructed_eeg_data)
        print(reconstructed_eeg_data.shape)
        reconstructed_eeg_data = mne.EpochsArray(reconstructed_eeg_data, info, verbose=False)

        # plot evoked topomap
        print(subject)
        times = np.linspace(0.0, 0.496, 10)
        evoked = X.average().pick('eeg')
        evoked.plot_topomap(times)
        reconstructed_evoked = reconstructed_eeg_data.average().pick('eeg')
        reconstructed_evoked.plot_topomap(times)

        del stc, reconstructed_eeg_data, X, Y
        gc.collect()

In [ ]:
def my_mask(x):
  return tf.cast(tf.greater_equal(x, 1), tf.float32)

def diff_mask(mask_op):
  @tf.custom_gradient
  def _diff_mask(x):
    def grad(dy):
      return dy * tf.ones_like(x)
    return mask_op(x), grad
  return _diff_mask

"""
Total params: 
Trainable params: 
Non-trainable params: 
"""
class AutoSelect(tf.keras.Model):
    def __init__(self, forward_matrix, random_select, use_mask, model_name='default'):
        super(AutoSelect, self).__init__()
        # preprocessing
        self.forward_matrix = tf.transpose(tf.constant(forward_matrix), perm=[1, 0])
        self.dropout = Dropout(0.1)
        self.source_select = Dense(forward_matrix.shape[1], activation=None)
        self.sigmoid = tf.keras.layers.Activation('sigmoid')
        self.model_name = model_name
        self.random_select = random_select
        self.use_mask = use_mask
        self.mask = tf.Variable(np.ones((1, forward_matrix.shape[1])), dtype=tf.float32)
        #self.mask = tf.Variable(np.random.rand(1, forward_matrix.shape[1])+0.5, dtype=tf.float32)
        
        # classifier
        self.conv1 = Conv1D(filters=200, kernel_size=3, strides=1, padding='same')
        self.bn1 = BatchNormalization()
        self.dropout1 = Dropout(0.5)
        self.relu1 = ReLU()
        self.flatten = Flatten()
        self.dense1 = Dense(6, activation="softmax")
        
        # eegnet
        if model_name == "eegnet":
            self.eegnet_model = tf.keras.models.Sequential([
                Conv2D(16, (1, 8), use_bias = False, activation = 'linear', padding='same', name = 'Spectral_filter'),
                BatchNormalization(),
                DepthwiseConv2D((124, 1), use_bias = False, padding='valid', depth_multiplier = 2, activation = 'linear',
                depthwise_constraint = tf.keras.constraints.MaxNorm(max_value=1), name = 'Spatial_filter'),
                BatchNormalization(),
                Activation('elu'),
                AveragePooling2D((1, 2)),
                Dropout(0.5),
                SeparableConv2D(32, (1, 4), use_bias = False, activation = 'linear', padding = 'same'),
                BatchNormalization(),
                Activation('elu'),
                AveragePooling2D((1, 2)),
                Dropout(0.5),
                Flatten(),
                Dense(6, activation = 'softmax', kernel_constraint = max_norm(0.25))
            ])

    def call(self, inputs):                                   # (n, X, 32)
        # preprocessing
        x = tf.transpose(inputs, perm=[0, 2, 1])              # (n, 32, X)
            
        if self.random_select:
            x = self.dropout(x)                               # (n, 32, X)
        else:
            if self.use_mask:
                #tf.print(self.mask)
                mask = diff_mask(my_mask)(self.mask)          # (1, X)
                x = x * mask                                  # (n, 32, X)                  
            else:
                source_select = self.source_select(x)         # (n, 32, X)
                source_select = self.dropout(source_select)   
                source_select = self.sigmoid(source_select)
                x = x * source_select                         # (n, 32, X)
        
        x = tf.matmul(x, self.forward_matrix)                 # (n, 32, 124)
        #print(x)
        
        # classifier
        if self.model_name == "default":
            x = self.conv1(x)
            x = self.bn1(x)
            x = self.dropout1(x)
            x = self.relu1(x)
            x = self.flatten(x)
            x = self.dense1(x)
        elif self.model_name == "eegnet":
            x = tf.transpose(x, [0, 2, 1])
            x = tf.expand_dims(x, axis=-1)                    # (n, 124, 32, 1)
            x = self.eegnet_model(x)
        
        return x

In [ ]:
data_path = "Shared drives/Motor Imagery/Visual Dataset"
    
epochs = create_epochs(data_path)
print(epochs.keys())

In [ ]:
#apply_inverse_and_forward_kfold(epochs, n_splits=n_splits, save_inverse=True, save_forward=True)
#plot_evoked_topomap(epochs)

# Classification (Original Data)

In [ ]:
"""
labels
1=Human Body; 2=Human Face; 3=Animal Body; 4=Animal Face; 5=Fruit Vegetable; 6=Inanimate Object

channels
124
"""

results = {"S1": {}, "S2": {}, "S3": {}, "S4": {}, "S5": {}, "S6": {}, "S7": {}, "S8": {}, "S9": {}, "S10": {}}
debug = True
training = True
model_name = "eegnet"

# train model on each subject individually
data_list = []
for subject in results.keys():
  data_list.append(subject)

# train model on individual subject
# data_list = []
# data_list.append("S1")

for data_name in data_list:
  accuracy = 0
  precision = 0
  recall = 0
  f1 = 0
  kappa = 0
  Confusion_matrix = []

  X = epochs[data_name]["epochs"].get_data()
  Y = epochs[data_name]["labels"]
  X = np.array(X)
  Y = np.array(Y)
    
  skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
  for train_index, test_index in skf.split(X, Y):
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]
    
    Y_train -= 1
    Y_test -= 1
    
    X_train = np.swapaxes(X_train, 1, 2)
    X_test = np.swapaxes(X_test, 1, 2)
    
#     X_train_shape = X_train.shape
#     X_test_shape = X_test.shape
#     X_train = X_train.reshape(X_train_shape[0], -1)
#     X_test = X_test.reshape(X_test_shape[0], -1)
#     print(X_train.shape, X_test.shape)
    
#     scaler = StandardScaler()
#     X_train = scaler.fit_transform(X_train)
#     X_test = scaler.transform(X_test)
#     X_train = X_train.reshape(X_train_shape)
#     X_test = X_test.reshape(X_test_shape)
#     print(X_train.shape, X_test.shape)
    
    if model_name == "eegnet":
        X_train = np.swapaxes(X_train, 1, 2)
        X_test = np.swapaxes(X_test, 1, 2)
        X_train = np.expand_dims(X_train, axis=-1)
        X_test = np.expand_dims(X_test, axis=-1)

    if debug:
      print(data_name)
      print("shape of X_train and Y_train: " + str(X_train.shape) + " " + str(Y_train.shape))
      print("shape of X_test and Y_test: " + str(X_test.shape) + " " + str(Y_test.shape))

    if training:
      # create new model
      model = create_model(model_name=model_name)
      
      log_dir = DIRECTORY_PATH + "/visual/logs/" + data_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
      optimizer = Adam(learning_rate=1e-5)
      model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
      model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1000, callbacks=[tensorboard_callback], verbose=0)

      Y_hat = model.predict(X_test)
      Y_hat = np.argmax(Y_hat, axis=1)
      accuracy += accuracy_score(Y_test, Y_hat)
      precision += precision_score(Y_test, Y_hat, average="macro")
      recall += recall_score(Y_test, Y_hat, average="macro")
      f1 += f1_score(Y_test, Y_hat, average="macro")
      kappa += cohen_kappa_score(Y_test, Y_hat)
      Confusion_matrix.append(confusion_matrix(Y_test, Y_hat, labels=range(6)))
        
      # save model
      model.save_weights(DIRECTORY_PATH + "/visual/models/" + data_name + "_" + str(accuracy_score(Y_test, Y_hat))[:6] + "/")
    else:
      # load pretrained model
      model = create_model(model_name=model_name)
      model.load_weights(DIRECTORY_PATH + "/visual/models/" + "A09_0.9183/")
      # freeze model
      model.trainable = False
      optimizer = Adam(learning_rate=1e-5)
      model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
      Y_hat = model.predict(X_test)
      Y_hat = np.argmax(Y_hat, axis=1)
      accuracy += accuracy_score(Y_test, Y_hat)
      precision += precision_score(Y_test, Y_hat, average="macro")
      recall += recall_score(Y_test, Y_hat, average="macro")
      f1 += f1_score(Y_test, Y_hat, average="macro")
      kappa += cohen_kappa_score(Y_test, Y_hat)
      Confusion_matrix.append(confusion_matrix(Y_test, Y_hat, labels=range(6)))

  accuracy /= n_splits
  precision /= n_splits
  recall /= n_splits
  f1 /= n_splits
  kappa /= n_splits
  if debug:
    print("accuracy: " + str(accuracy))
    print("precision: " + str(precision))
    print("recall: " + str(recall))
    print("f1: " + str(f1))
    print("kappa: " + str(kappa))
    
    if not os.path.exists(DIRECTORY_PATH + "/visual/pics"):
      os.mkdir(DIRECTORY_PATH + "/visual/pics")

    for i in range(len(Confusion_matrix)):
      disp = ConfusionMatrixDisplay(confusion_matrix=Confusion_matrix[i], display_labels=range(6))
      disp.plot()
      plt.savefig(DIRECTORY_PATH + "/visual/pics/"+data_name+"_"+str(i)+'_confusion_matrix.png', bbox_inches='tight')
      plt.show()

  results[data_name]["accuracy"] = accuracy
  results[data_name]["precision"] = precision
  results[data_name]["recall"] = recall
  results[data_name]["f1"] = f1
  results[data_name]["kappa"] = kappa

In [ ]:
# Calculate average performance
average_accuracy = 0
average_precision = 0
average_recall = 0
average_f1 = 0
average_kappa = 0
for key, value in results.items():
  average_accuracy += value["accuracy"]
  average_precision += value["precision"]
  average_recall += value["recall"]
  average_f1 += value["f1"]
  average_kappa += value["kappa"]

average_accuracy /= 10
average_precision /= 10
average_recall /= 10
average_f1 /= 10
average_kappa /= 10

print("average accuracy: " + str(average_accuracy))
print("average precision: " + str(average_precision))
print("average recall: " + str(average_recall))
print("average f1: " + str(average_f1))
print("average kappa: " + str(average_kappa))

In [ ]:
# time computation
debug = False
model_name = "default"
warm_up = 10 # initializing memory allocators, and GPU-related initializations 

data_list = []
data_list.append("S1")

for data_name in data_list:
  X = epochs[data_name]["epochs"].get_data()
  Y = epochs[data_name]["labels"]
  X = np.array(X)
  Y = np.array(Y)
    
  Y -= 1
  X = np.swapaxes(X, 1, 2)
    
  if model_name == "eegnet":
    X = np.swapaxes(X, 1, 2)
    X = np.expand_dims(X, axis=-1)
    
  X_test = np.expand_dims(X[0], axis=0)
  Y_test = np.expand_dims(Y[0], axis=0)
  print(X_test.shape)
    
  # load pretrained model
  model = create_model(model_name=model_name)
  model.load_weights("D:/forward and inverse results (new)/visual/default/original EEG/models/S1_0.4720/")
  # freeze model
  model.trainable = False
  optimizer = Adam(learning_rate=1e-5)
  model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  if model_name == "default":
    model.build(input_shape=(None, 32, 124))
  elif model_name == "eegnet":
    model.build(input_shape=(None, 124, 32, 1))
  print(model.summary())

  for i in range(warm_up):
    if i == warm_up-1:
        start = time.time()
    
    Y_hat = model.predict(X_test)
    
    if i == warm_up-1:
        end = time.time()
        print("time used: ", (end - start)*1000, "ms")

# Classification (Reconstruct EEG)

In [ ]:
"""
labels
1=Human Body; 2=Human Face; 3=Animal Body; 4=Animal Face; 5=Fruit Vegetable; 6=Inanimate Object

channels
124
"""

results = {"S1": {}, "S2": {}, "S3": {}, "S4": {}, "S5": {}, "S6": {}, "S7": {}, "S8": {}, "S9": {}, "S10": {}}
debug = True
training = True
saving_directory = "/v1-v5 + IT, MT, ST, F reconstruct"
model_name = "eegnet"

# train model on each subject individually
data_list = []
for subject in results.keys():
  data_list.append(subject)

# train model on individual subject
# data_list = []
# data_list.append("S9")
# data_list.append("S10")

for data_name in data_list:
  # load data from external storage
  directory_path = op.join(EXTERNAL_STORAGE_PATH, "v1-v5 + IT, MT, ST, F region", "data", "reconstructed eeg", data_name)
  counter = 0
  accuracy = 0
  precision = 0
  recall = 0
  f1 = 0
  kappa = 0
  Confusion_matrix = []

  while(counter < n_splits):
    counter += 1
    X_train = np.load(op.join(directory_path, str(counter)+"_train_X.npz"), allow_pickle=True)["data"]
    X_test = np.load(op.join(directory_path, str(counter)+"_test_X.npz"), allow_pickle=True)["data"]
    Y_train = np.load(op.join(directory_path, str(counter)+"_train_Y.npz"), allow_pickle=True)["data"]
    Y_test = np.load(op.join(directory_path, str(counter)+"_test_Y.npz"), allow_pickle=True)["data"]
    
    Y_train -= 1
    Y_test -= 1
    
    X_train = np.swapaxes(X_train, 1, 2)
    X_test = np.swapaxes(X_test, 1, 2)
    
    if model_name == "eegnet":
        X_train = np.swapaxes(X_train, 1, 2)
        X_test = np.swapaxes(X_test, 1, 2)
        X_train = np.expand_dims(X_train, axis=-1)
        X_test = np.expand_dims(X_test, axis=-1)

    if debug:
      print(data_name)
      print("shape of X_train and Y_train: " + str(X_train.shape) + " " + str(Y_train.shape))
      print("shape of X_test and Y_test: " + str(X_test.shape) + " " + str(Y_test.shape))

    if training:
      # create new model
      model = create_model(model_name=model_name)
      
      log_dir = DIRECTORY_PATH + saving_directory +"/logs/" + data_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
      tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
      optimizer = Adam(learning_rate=1e-5)
      model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
      model.fit(X_train, Y_train, validation_data=(X_test, Y_test), batch_size=32, epochs=1000, callbacks=[tensorboard_callback], verbose=0)

      Y_hat = model.predict(X_test)
      Y_hat = np.argmax(Y_hat, axis=1)
      accuracy += accuracy_score(Y_test, Y_hat)
      precision += precision_score(Y_test, Y_hat, average="macro")
      recall += recall_score(Y_test, Y_hat, average="macro")
      f1 += f1_score(Y_test, Y_hat, average="macro")
      kappa += cohen_kappa_score(Y_test, Y_hat)
      Confusion_matrix.append(confusion_matrix(Y_test, Y_hat, labels=range(6)))

      # save model
      model.save_weights(DIRECTORY_PATH + saving_directory + "/models/" + data_name + "_" + str(accuracy_score(Y_test, Y_hat))[:6] + "/")
    else:
      # load pretrained model
      model = create_model(model_name=model_name)
      model.load_weights(DIRECTORY_PATH + saving_directory + "/models/" + "A09_0.9183/")
      # freeze model
      model.trainable = False
      optimizer = Adam(learning_rate=1e-5)
      model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
      Y_hat = model.predict(X_test)
      Y_hat = np.argmax(Y_hat, axis=1)
      accuracy += accuracy_score(Y_test, Y_hat)
      precision += precision_score(Y_test, Y_hat, average="macro")
      recall += recall_score(Y_test, Y_hat, average="macro")
      f1 += f1_score(Y_test, Y_hat, average="macro")
      kappa += cohen_kappa_score(Y_test, Y_hat)
      Confusion_matrix.append(confusion_matrix(Y_test, Y_hat, labels=range(6)))

  accuracy /= n_splits
  precision /= n_splits
  recall /= n_splits
  f1 /= n_splits
  kappa /= n_splits
  if debug:
    print("accuracy: " + str(accuracy))
    print("precision: " + str(precision))
    print("recall: " + str(recall))
    print("f1: " + str(f1))
    print("kappa: " + str(kappa))
    
    if not os.path.exists(DIRECTORY_PATH + saving_directory + "/pics"):
      os.mkdir(DIRECTORY_PATH + saving_directory + "/pics")

    for i in range(len(Confusion_matrix)):
      disp = ConfusionMatrixDisplay(confusion_matrix=Confusion_matrix[i], display_labels=range(6))
      disp.plot()
      plt.savefig(DIRECTORY_PATH + saving_directory + "/pics/"+data_name+"_"+str(i)+'_confusion_matrix.png', bbox_inches='tight')
      plt.show()

  results[data_name]["accuracy"] = accuracy
  results[data_name]["precision"] = precision
  results[data_name]["recall"] = recall
  results[data_name]["f1"] = f1
  results[data_name]["kappa"] = kappa

In [ ]:
# Calculate average performance
average_accuracy = 0
average_precision = 0
average_recall = 0
average_f1 = 0
average_kappa = 0
for key, value in results.items():
  average_accuracy += value["accuracy"]
  average_precision += value["precision"]
  average_recall += value["recall"]
  average_f1 += value["f1"]
  average_kappa += value["kappa"]

average_accuracy /= 10
average_precision /= 10
average_recall /= 10
average_f1 /= 10
average_kappa /= 10

print("average accuracy: " + str(average_accuracy))
print("average precision: " + str(average_precision))
print("average recall: " + str(average_recall))
print("average f1: " + str(average_f1))
print("average kappa: " + str(average_kappa))

In [ ]:
# time computation
debug = False
model_name = "eegnet"
warm_up = 10 # initializing memory allocators, and GPU-related initializations 

data_list = []
data_list.append("S1")

information = get_inverse_and_forward_information(epochs)
print(information)
leadfield = information["leadfield"]
inverse_operator = information["inverse_operator"]
my_left_points = information["my_left_points"]
my_right_points = information["my_right_points"]
info = epochs["S1"]["epochs"].info

for data_name in data_list:
  X = epochs[data_name]["epochs"].get_data()
  Y = epochs[data_name]["labels"]
  X = np.array(X)
  Y = np.array(Y)
    
  Y -= 1
  X_test = np.expand_dims(X[0], axis=0)
  Y_test = np.expand_dims(Y[0], axis=0)
  print(X_test.shape)
    
  # load pretrained model
  model = create_model(model_name=model_name)
  model.load_weights("D:/forward and inverse results (new)/visual/eegnet/v1-v5 reconstruct/models/S1_0.4489/")
  # freeze model
  model.trainable = False
  optimizer = Adam(learning_rate=1e-5)
  model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  if model_name == "default":
    model.build(input_shape=(None, 32, 124))
  elif model_name == "eegnet":
    model.build(input_shape=(None, 124, 32, 1))

  for i in range(warm_up):
    if i == warm_up-1:
        start = time.time()
        
    X_epochs = mne.EpochsArray(X_test, info, verbose=False)
    method = "sLORETA"
    snr = 3.
    lambda2 = 1. / snr ** 2
    stc_test = apply_inverse_epochs(X_epochs, inverse_operator, lambda2,
                                  method=method, pick_ori="normal", verbose=debug)
    
    # slice reconstructed eeg data
    reconstructed_eeg_data = []
    for source in stc_test:
        visual_source = np.zeros_like(source.data)
        visual_source[:len(source.vertices[0])][my_left_points] = source.data[:len(source.vertices[0])][my_left_points]
        visual_source[-len(source.vertices[1]):][my_right_points] = source.data[-len(source.vertices[1]):][my_right_points]
        visual_eeg = np.dot(leadfield, visual_source)
        reconstructed_eeg_data.append(visual_eeg)
    reconstructed_eeg_data = np.array(reconstructed_eeg_data)
    print(reconstructed_eeg_data.shape)
    
    X_time = np.swapaxes(reconstructed_eeg_data, 1, 2)
    
    if model_name == "eegnet":
      X_time = np.swapaxes(X_time, 1, 2)
      X_time = np.expand_dims(X_time, axis=-1)
    
    Y_hat = model.predict(X_time)
    
    if i == warm_up-1:
        end = time.time()
        print("time used: ", (end - start)*1000, "ms")

# Classification (Auto-select Source Activity)

In [ ]:
information = get_inverse_and_forward_information(epochs)
print(information)

print(information["leadfield"][:, :len(information["left_vertices"])][:, information["my_left_points"]].shape)
print(information["leadfield"][:, -len(information["right_vertices"]):][:, information["my_right_points"]].shape)
forward_matrix = np.concatenate((information["leadfield"][:, :len(information["left_vertices"])][:, information["my_left_points"]], 
                          information["leadfield"][:, -len(information["right_vertices"]):][:, information["my_right_points"]]),
                          axis = 1)
print(forward_matrix.shape)    

In [ ]:
"""
labels
1=Human Body; 2=Human Face; 3=Animal Body; 4=Animal Face; 5=Fruit Vegetable; 6=Inanimate Object

channels
124
"""

results = {"S1": {}, "S2": {}, "S3": {}, "S4": {}, "S5": {}, "S6": {}, "S7": {}, "S8": {}, "S9": {}, "S10": {}}
debug = True
training = True
random_select = False
use_mask = True
saving_directory = "/v1-v5 source selection initialize mask 1"
epochs = 1000
batch_size = 32
model_name = "eegnet"

# train model on each subject individually
data_list = []
for subject in results.keys():
  data_list.append(subject)

# train model on individual subject
# data_list = []
# data_list.append("S2")

for data_name in data_list:
  # load data from external storage
  directory_path = op.join(EXTERNAL_STORAGE_PATH, "v1-v5 region", "data", "source activity", data_name)
  counter = 0
  accuracy = 0
  precision = 0
  recall = 0
  f1 = 0
  kappa = 0
  Confusion_matrix = []

  while(counter < n_splits):
        
    counter += 1
    X_train = np.load(op.join(directory_path, str(counter)+"_train_X.npz"), allow_pickle=True)["data"]
    X_test = np.load(op.join(directory_path, str(counter)+"_test_X.npz"), allow_pickle=True)["data"]
    Y_train = np.load(op.join(directory_path, str(counter)+"_train_Y.npz"), allow_pickle=True)["data"]
    Y_test = np.load(op.join(directory_path, str(counter)+"_test_Y.npz"), allow_pickle=True)["data"]
    
    Y_train -= 1
    Y_test -= 1
    
    X_train_batches = np.array_split(X_train, X_train.shape[0] // batch_size + 1)
    Y_train_batches = np.array_split(Y_train, X_train.shape[0] // batch_size + 1)
    X_test_batches = np.array_split(X_test, X_test.shape[0] // batch_size + 1)
    Y_test_batches = np.array_split(Y_test, Y_test.shape[0] // batch_size + 1)

    if debug:
      print(data_name)
      print("shape of X_train and Y_train: " + str(X_train.shape) + " " + str(Y_train.shape))
      print("shape of X_test and Y_test: " + str(X_test.shape) + " " + str(Y_test.shape))

    if training:
      # create new model
      model = AutoSelect(forward_matrix, random_select, use_mask, model_name)
      
      log_dir = DIRECTORY_PATH + saving_directory +"/logs/" + data_name + "/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
      optimizer = Adam(learning_rate=1e-5)
      loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
      train_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='train_acc')
      test_acc_metric = tf.keras.metrics.SparseCategoricalAccuracy(name='test_acc')
      train_loss = tf.keras.metrics.Mean('train_loss', dtype=tf.float32)
      test_loss = tf.keras.metrics.Mean('test_loss', dtype=tf.float32)
      train_log_writer = tf.summary.create_file_writer(log_dir+"/train")
      test_log_writer = tf.summary.create_file_writer(log_dir+"/test")
    
      @tf.function
      def train_step(x, y):
        with tf.GradientTape() as tape:
            prediction = model(x, training=True)
            loss_value = loss_fn(y, prediction)
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))
        train_acc_metric.update_state(y, prediction)
        train_loss.update_state(loss_value)
        return loss_value

      @tf.function
      def test_step(x, y):
        test_prediction = model(x, training=False)
        loss_value = loss_fn(y, test_prediction)
        test_acc_metric.update_state(y, test_prediction)
        test_loss.update_state(loss_value)
        
      for epoch in range(epochs):
        #print("\nStart of epoch %d" % (epoch,))
        start_time = time.time()

        for batch_index in range(len(X_train_batches)):
            loss_value = train_step(X_train_batches[batch_index], Y_train_batches[batch_index])
        with train_log_writer.as_default():
            tf.summary.scalar('loss', train_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', train_acc_metric.result(), step=epoch)
        #print("Training loss at epoch %d: %.4f" % (epoch, float(loss_value)))
        train_acc = train_acc_metric.result()
        #print("Training acc over epoch: %.4f" % (float(train_acc),))
        train_acc_metric.reset_states()
        train_loss.reset_states()

        for batch_index in range(len(X_test_batches)):
            test_step(X_test_batches[batch_index], Y_test_batches[batch_index])
        with test_log_writer.as_default():
            tf.summary.scalar('loss', test_loss.result(), step=epoch)
            tf.summary.scalar('accuracy', test_acc_metric.result(), step=epoch)
        test_acc = test_acc_metric.result()
        test_acc_metric.reset_states()
        test_loss.reset_states()
        #print("Test acc: %.4f" % (float(test_acc),))
        #print("Time taken: %.2fs" % (time.time() - start_time))

      Y_hat = model(X_test, training=False)
      Y_hat = np.argmax(Y_hat, axis=1)
      accuracy += accuracy_score(Y_test, Y_hat)
      precision += precision_score(Y_test, Y_hat, average="macro")
      recall += recall_score(Y_test, Y_hat, average="macro")
      f1 += f1_score(Y_test, Y_hat, average="macro")
      kappa += cohen_kappa_score(Y_test, Y_hat)
      Confusion_matrix.append(confusion_matrix(Y_test, Y_hat, labels=range(6)))

      # save model
      model.save_weights(DIRECTORY_PATH + saving_directory + "/models/" + data_name + "_" + str(accuracy_score(Y_test, Y_hat))[:6] + "/")
    else:
      # load pretrained model
      model = AutoSelect(forward_matrix, random_select, use_mask, model_name)
      model.load_weights(DIRECTORY_PATH + saving_directory + "/models/" + "A09_0.9183/")
      # freeze model
      model.trainable = False
      optimizer = Adam(learning_rate=1e-5)
      model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        
      Y_hat = model(X_test, training=False)
      Y_hat = np.argmax(Y_hat, axis=1)
      accuracy += accuracy_score(Y_test, Y_hat)
      precision += precision_score(Y_test, Y_hat, average="macro")
      recall += recall_score(Y_test, Y_hat, average="macro")
      f1 += f1_score(Y_test, Y_hat, average="macro")
      kappa += cohen_kappa_score(Y_test, Y_hat)
      Confusion_matrix.append(confusion_matrix(Y_test, Y_hat, labels=range(6)))

  accuracy /= n_splits
  precision /= n_splits
  recall /= n_splits
  f1 /= n_splits
  kappa /= n_splits
  if debug:
    print("accuracy: " + str(accuracy))
    print("precision: " + str(precision))
    print("recall: " + str(recall))
    print("f1: " + str(f1))
    print("kappa: " + str(kappa))
    
    if not os.path.exists(DIRECTORY_PATH + saving_directory + "/pics"):
      os.mkdir(DIRECTORY_PATH + saving_directory + "/pics")

    for i in range(len(Confusion_matrix)):
      disp = ConfusionMatrixDisplay(confusion_matrix=Confusion_matrix[i], display_labels=range(6))
      disp.plot()
      plt.savefig(DIRECTORY_PATH + saving_directory + "/pics/"+data_name+"_"+str(i)+'_confusion_matrix.png', bbox_inches='tight')
      plt.show()

  results[data_name]["accuracy"] = accuracy
  results[data_name]["precision"] = precision
  results[data_name]["recall"] = recall
  results[data_name]["f1"] = f1
  results[data_name]["kappa"] = kappa

In [ ]:
# Calculate average performance
average_accuracy = 0
average_precision = 0
average_recall = 0
average_f1 = 0
average_kappa = 0
for key, value in results.items():
  average_accuracy += value["accuracy"]
  average_precision += value["precision"]
  average_recall += value["recall"]
  average_f1 += value["f1"]
  average_kappa += value["kappa"]

average_accuracy /= 10
average_precision /= 10
average_recall /= 10
average_f1 /= 10
average_kappa /= 10

print("average accuracy: " + str(average_accuracy))
print("average precision: " + str(average_precision))
print("average recall: " + str(average_recall))
print("average f1: " + str(average_f1))
print("average kappa: " + str(average_kappa))

In [ ]:
# time computation
debug = False
random_select = False
use_mask = True
model_name = "eegnet"
warm_up = 10 # initializing memory allocators, and GPU-related initializations 

data_list = []
data_list.append("S1")

information = get_inverse_and_forward_information(epochs)
print(information)
print(information["leadfield"][:, :len(information["left_vertices"])][:, information["my_left_points"]].shape)
print(information["leadfield"][:, -len(information["right_vertices"]):][:, information["my_right_points"]].shape)
forward_matrix = np.concatenate((information["leadfield"][:, :len(information["left_vertices"])][:, information["my_left_points"]], 
                          information["leadfield"][:, -len(information["right_vertices"]):][:, information["my_right_points"]]),
                          axis = 1)
print(forward_matrix.shape)
inverse_operator = information["inverse_operator"]
my_left_points = information["my_left_points"]
my_right_points = information["my_right_points"]
info = epochs["S1"]["epochs"].info

for data_name in data_list:
  X = epochs[data_name]["epochs"].get_data()
  Y = epochs[data_name]["labels"]
  X = np.array(X)
  Y = np.array(Y)
    
  Y -= 1
  X_test = np.expand_dims(X[0], axis=0)
  Y_test = np.expand_dims(Y[0], axis=0)
  print(X_test.shape)
    
  # load pretrained model
  model = AutoSelect(forward_matrix, random_select, use_mask, model_name)
  model.load_weights("D:/forward and inverse results (new)/visual/eegnet/v1-v5+IT,MT,ST,F source selection Mask Initialize 1/models/S1_0.4720/")
  # freeze model
  model.trainable = False
  optimizer = Adam(learning_rate=1e-5)
  model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
  model.build(input_shape=(None, forward_matrix.shape[1], 32))

  for i in range(warm_up):
    if i == warm_up-1:
        start = time.time()
        
    X_epochs = mne.EpochsArray(X_test, info, verbose=False)
    method = "sLORETA"
    snr = 3.
    lambda2 = 1. / snr ** 2
    stc_test = apply_inverse_epochs(X_epochs, inverse_operator, lambda2,
                                  method=method, pick_ori="normal", verbose=debug)
    
    # slice source activity data
    left_hemi_data = []
    right_hemi_data = []
    for source in stc_test:
        left_hemi_data.append(source.data[:len(source.vertices[0])][my_left_points])
        right_hemi_data.append(source.data[-len(source.vertices[1]):][my_right_points])
    left_hemi_data = np.array(left_hemi_data)
    right_hemi_data = np.array(right_hemi_data)
    
    X_time = np.append(left_hemi_data, right_hemi_data, axis=1)
    print(X_time.shape)
    
    Y_hat = model(X_time, training=False)
    
    if i == warm_up-1:
        end = time.time()
        print("time used: ", (end - start)*1000, "ms")

# Visualizing Mask and Attention

In [ ]:
model = AutoSelect(forward_matrix, False, False, "default")
model.build(input_shape=(None, 3240, 32))
#print(model.summary())
load_model_directory = "v1-v5 source selection initialize mask 1/models/"
load_model_subject = "S6_0.6470/"

model.load_weights("D:" + "/forward and inverse results (new)/visual/default/" + load_model_directory + load_model_subject)

In [ ]:
for weight in model.get_weights():
    print(weight.shape)

In [ ]:
mni_lh = mne.vertex_to_mni(information["left_vertices"], 0, mne_subject)
print(mni_lh.shape)
mni_rh = mne.vertex_to_mni(information["right_vertices"], 1, mne_subject)
print(mni_rh.shape)

mni_left_points = mni_lh[information["my_left_points"]]
print(mni_left_points.shape)
mni_right_points = mni_rh[information["my_right_points"]]
print(mni_right_points.shape)

fig = plt.figure(figsize=(8, 8))
ax = fig.add_subplot(projection='3d')
ax.scatter(mni_lh[:, 0], mni_lh[:, 1], mni_lh[:, 2], s=15, marker='|')
ax.scatter(mni_rh[:, 0], mni_rh[:, 1], mni_rh[:, 2], s=15, marker='_')
ax.scatter(mni_left_points[:, 0], mni_left_points[:, 1], mni_left_points[:, 2], s=15, marker='o')
ax.scatter(mni_right_points[:, 0], mni_right_points[:, 1], mni_right_points[:, 2], s=15, marker='^')
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

## Mask Visualization

In [ ]:
# for mask
#np.sum(model.get_weights()[-1] >= 1)

for pts in model.get_weights()[-1]:
    print(pts)
print(np.max(model.get_weights()[-1]))
print(np.min(model.get_weights()[-1]))
print(np.mean(model.get_weights()[-1]))
print(np.std(model.get_weights()[-1]))

In [ ]:
left_weight = np.zeros((information["my_left_points"].shape))
right_weight =  np.zeros((information["my_right_points"].shape))
left_weight[information["my_left_points"]] = model.get_weights()[-1].reshape(-1)[:np.sum(information["my_left_points"])]
right_weight[information["my_right_points"]] = model.get_weights()[-1].reshape(-1)[np.sum(information["my_left_points"]):]

total_weight = np.append(left_weight, right_weight, axis=0)
print(total_weight)
print(total_weight.shape)

# binary weights
total_weight = (total_weight >= 1).astype(int)
print(total_weight)

In [ ]:
vertices_tree = KDTree(mm_coords.reshape(-1, 3))
final_weight = np.zeros(mm_coords.reshape(-1, 3).shape)

max_points = 0
for i, vertex_i in enumerate(vertices_tree.query_ball_point(np.append(mni_lh, mni_rh, axis=0), 2)):
    if vertex_i != []:
        if np.max(vertex_i) > max_points:
            max_points = np.max(vertex_i)
        #print(mm_coords.reshape(-1, 3)[np.max(vertex_i)])
        #print(np.append(mni_lh, mni_rh, axis=0)[i])
        
        # fill all
        for vertex_j in vertex_i:
            final_weight[vertex_j] = total_weight[i]
        # fill biggest index
        #final_weight[np.max(vertex_i)] = total_weight[i]
        #print(vertex_i)
print(max_points)

final_weight = final_weight.reshape(mm_coords.shape)[:, :, :, 0]

visualization_mask = Nifti1Image(final_weight, ch2_img.affine, ch2_img.header)
nib.save(visualization_mask, os.path.join('C:/Users/ivanlim/Desktop/EEG-forward-and-inverse', 'visualization_mask.nii.gz'))  

In [ ]:
visualization_mask_with_shape = Nifti1Image(np.asanyarray(final_weight), ch2_img.affine, ch2_img.header)
nib.save(visualization_mask_with_shape, os.path.join('C:/Users/ivanlim/Desktop/EEG-forward-and-inverse', 'visualization_mask_with_shape.nii.gz')) 

In [ ]:
plotting.plot_glass_brain("C:/Users/ivanlim/Desktop/EEG-forward-and-inverse/visualization_mask_with_shape.nii.gz", display_mode='xz', threshold=0.9)

# Statistical test

In [ ]:
def parse_tensorboard(path, scalars):
    """returns a dictionary of pandas dataframes for each requested scalar"""
    ea = event_accumulator.EventAccumulator(
        path,
        size_guidance={event_accumulator.SCALARS: 0},
    )
    _absorb_print = ea.Reload()
    #print(ea.Tags())
    # make sure the scalars are in the event accumulator tags
    assert all(
        s in ea.Tags()['tensors'] for s in scalars
    ), "some scalars were not found in the event accumulator"
    return {k: pd.DataFrame(ea.Tensors(k)) for k in scalars}

In [ ]:
method_path = "D:/forward and inverse results (new)/visual/default"
accuracy_dict = {}

for method in ["original EEG", "v1-v5 reconstruct", "v1-v5 + IT, MT, ST, F reconstruct", 
               "v1-v5 source selection dense + dropout 0.1", "v1-v5 source selection initialize mask 1",
               "v1-v5 source selection mask initialize 0.5-1.5", "v1-v5+IT,MT,ST,F source selection Mask Initialize 1",
               "v1-v5+IT,MT,ST,F source selection Mask initialize 0.5-1.5"]:
    logs_path = os.path.join(method_path, method, "logs")
    if not accuracy_dict.get(method):
        accuracy_dict[method] = {}
    for subject in os.listdir(logs_path):
        if not accuracy_dict[method].get(subject):
            accuracy_dict[method][subject] = {}
        subject_path = os.path.join(logs_path, subject)
        for time in os.listdir(subject_path):
            if method == "original EEG" or method == "v1-v5 reconstruct" or method == "v1-v5 + IT, MT, ST, F reconstruct" or method == "v1-v5 source selection initialize mask 1":
                validation_path = os.path.join(subject_path, time, "validation")
            elif method == "v1-v5 source selection mask initialize 0.5-1.5":
                validation_path = os.path.join(subject_path, time)
            else:
                validation_path = os.path.join(subject_path, time, "test")
            tensorboard_logs = os.path.join(validation_path, os.listdir(validation_path)[0])
            if method == "original EEG" or method == "v1-v5 reconstruct" or method == "v1-v5 + IT, MT, ST, F reconstruct" or method == "v1-v5 source selection initialize mask 1":
                df = parse_tensorboard(tensorboard_logs, ["epoch_loss", "epoch_accuracy"])
                last_test_accuracy = tf.constant(tf.make_ndarray(df["epoch_accuracy"]["tensor_proto"][len(df["epoch_accuracy"]["tensor_proto"])-1]))
            elif method == "v1-v5 source selection mask initialize 0.5-1.5":
                df = parse_tensorboard(tensorboard_logs, ["testing loss", "testing accuracy"])
                last_test_accuracy = tf.constant(tf.make_ndarray(df["testing accuracy"]["tensor_proto"][len(df["testing accuracy"]["tensor_proto"])-1]))
            else:
                df = parse_tensorboard(tensorboard_logs, ["loss", "accuracy"])
                last_test_accuracy = tf.constant(tf.make_ndarray(df["accuracy"]["tensor_proto"][len(df["accuracy"]["tensor_proto"])-1]))
            #print(subject, time)
            accuracy_dict[method][subject][time] = last_test_accuracy.numpy()
            
accuracy_mean_std = {}
for method in accuracy_dict.keys():
    accuracy_mean_std[method] = {}
    for subject in accuracy_dict[method].keys():
        accuracy_time = list(accuracy_dict[method][subject].values())
        mean = np.mean(accuracy_time)
        std = np.std(accuracy_time)
        accuracy_mean_std[method][subject] = {}
        accuracy_mean_std[method][subject]["mean"] = mean
        accuracy_mean_std[method][subject]["std"] = std
        #print(method, subject, mean)
        
subject_list = ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "S10"]
test_samples = [1038, 1037, 1037, 1037, 1037, 1037, 1038, 1037, 1037, 1037]
t_test_results = {}

for i, subject in enumerate(subject_list):
    t_test_results[subject] = {}
    count = 0
    for method in accuracy_mean_std.keys():
        if method == "original EEG" or method == "all motor attention with dropout 0.5":
            continue
        mean_1 = accuracy_mean_std["original EEG"][subject]["mean"]
        mean_2 = accuracy_mean_std[method][subject]["mean"]
        std_1 = accuracy_mean_std["original EEG"][subject]["std"]
        std_2 = accuracy_mean_std[method][subject]["std"]
        n = test_samples[i]
        
        df = n+n-2
        s_p = np.sqrt(((n-1)*(std_1**2)+(n-1)*(std_2**2))/(n+n-2))
        t = (mean_1-mean_2) / (s_p*np.sqrt(1/n+1/n))
        p_value = stats.t.sf(np.abs(t), df)*2

        if p_value < 0.05:
            count += 1
            print(subject, method, mean_2)
        #print(t, df, p_value)
    print(count)

In [ ]:
method_path = "D:/forward and inverse results (new)/visual/eegnet"
accuracy_dict = {}

for method in ["original EEG", "v1-v5 reconstruct", "v1-v5 + IT, MT, ST, F reconstruct", 
               "v1-v5 source selection dense + dropout 0.1", "v1-v5 source selection initialize mask 1",
               "v1-v5 source selection initialize mask 0.5 - 1.5", "v1-v5+IT,MT,ST,F source selection Mask Initialize 1",
               "v1-v5+IT,MT,ST,F source selection Mask initialize 0.5-1.5"]:
    logs_path = os.path.join(method_path, method, "logs")
    if not accuracy_dict.get(method):
        accuracy_dict[method] = {}
    for subject in os.listdir(logs_path):
        if not accuracy_dict[method].get(subject):
            accuracy_dict[method][subject] = {}
        subject_path = os.path.join(logs_path, subject)
        for time in os.listdir(subject_path):
            if method == "original EEG" or method == "v1-v5 reconstruct" or method == "v1-v5 + IT, MT, ST, F reconstruct":
                validation_path = os.path.join(subject_path, time, "validation")
            else:
                validation_path = os.path.join(subject_path, time, "test")
            tensorboard_logs = os.path.join(validation_path, os.listdir(validation_path)[0])
            if method == "original EEG" or method == "v1-v5 reconstruct" or method == "v1-v5 + IT, MT, ST, F reconstruct":
                df = parse_tensorboard(tensorboard_logs, ["epoch_loss", "epoch_accuracy"])
                last_test_accuracy = tf.constant(tf.make_ndarray(df["epoch_accuracy"]["tensor_proto"][len(df["epoch_accuracy"]["tensor_proto"])-1]))
            else:
                df = parse_tensorboard(tensorboard_logs, ["loss", "accuracy"])
                last_test_accuracy = tf.constant(tf.make_ndarray(df["accuracy"]["tensor_proto"][len(df["accuracy"]["tensor_proto"])-1]))
            #print(subject, time)
            accuracy_dict[method][subject][time] = last_test_accuracy.numpy()
            
accuracy_mean_std = {}
for method in accuracy_dict.keys():
    accuracy_mean_std[method] = {}
    for subject in accuracy_dict[method].keys():
        accuracy_time = list(accuracy_dict[method][subject].values())
        mean = np.mean(accuracy_time)
        std = np.std(accuracy_time)
        accuracy_mean_std[method][subject] = {}
        accuracy_mean_std[method][subject]["mean"] = mean
        accuracy_mean_std[method][subject]["std"] = std
        #print(method, subject, mean)
        
subject_list = ["S1", "S2", "S3", "S4", "S5", "S6", "S7", "S8", "S9", "S10"]
test_samples = [1038, 1037, 1037, 1037, 1037, 1037, 1038, 1037, 1037, 1037]
t_test_results = {}

for i, subject in enumerate(subject_list):
    t_test_results[subject] = {}
    count = 0
    for method in accuracy_mean_std.keys():
        if method == "original EEG" or method == "all motor attention with dropout 0.5":
            continue
        mean_1 = accuracy_mean_std["original EEG"][subject]["mean"]
        mean_2 = accuracy_mean_std[method][subject]["mean"]
        std_1 = accuracy_mean_std["original EEG"][subject]["std"]
        std_2 = accuracy_mean_std[method][subject]["std"]
        n = test_samples[i]
        
        df = n+n-2
        s_p = np.sqrt(((n-1)*(std_1**2)+(n-1)*(std_2**2))/(n+n-2))
        t = (mean_1-mean_2) / (s_p*np.sqrt(1/n+1/n))
        p_value = stats.t.sf(np.abs(t), df)*2

        if p_value < 0.05:
            count += 1
            print(subject, method, mean_2)
        #print(t, df, p_value)
    print(count)